# Remeber to update dictionary 

In [1]:
import httplib2
import json
import sys
import codecs
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np

In [2]:
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\N-Series.xlsx', dtype=str)

In [3]:
ME_df

,Make,Model,Year
0,Isuzu,N-Series,2000
1,Isuzu,N-Series,2001
2,Isuzu,N-Series,2002
3,Isuzu,N-Series,2003
4,Isuzu,N-Series,2004
5,Isuzu,N-Series,2005
6,Isuzu,N-Series,2006
7,Isuzu,N-Series,2007
8,Isuzu,N-Series,2008
9,Isuzu,N-Series,2009


In [4]:
dictionary = {542:'isuzu'}

In [41]:
def getMake():
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMakeID(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    MK_df = pd.DataFrame(result['Results'])
    result_df = MK_df[MK_df['Make_Name'].str.contains(Make)]
    return result_df

def getModel(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/%s?format=json' %Make)
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year(Make,Year):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeYear/make/%s/modelyear/%s?format=json' %(Make,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year_MakeID(MakeID,Year):
    url = ('https://vpic.nhtsa.dot.gov/api//vehicles/GetModelsForMakeIdYear/makeId/%s/modelyear/%s?format=json' %(MakeID,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMatch(Make_id,Year):
    Rlist =[]
    Valid_df = getModel_Year_MakeID(Make_id,Year)
    Rlist.append( list( Valid_df[Valid_df.Model_Name.str.lower().str.contains('n')].Model_Name))
    return Rlist
# With score
# def getExactMatch(Make_id,Year):
#     Rlist = []
#     try:
#         Valid_df = getModel_Year_MakeID(Make_id,Year)
        
#     except:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         Rlist=['No Such year in the Database'] *len(QC_Make_year)
#     else:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         for i in range(len(QC_Make_year.Model)):
#             str2Match = str(QC_Make_year.Model.values[i])
#             strOptions = [z for z in Valid_df.Model_Name]
#             Ratios= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
#             Rlist.append((':'.join([ str(a) for a in list(Ratios)])))
#     return Rlist

def getTopMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)
        
    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios= process.extractOne(str2Match,strOptions)
            Rlist.append(( list(Ratios)[0]))
    return Rlist

def getQCtable_OneYear(Make_id,Year):    
    QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
    QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
    QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
    QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
    QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
    QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') for i in range(len(QC_Make_Year['TopMatch']))]
    QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make_Year

def getQCtable_Allyear(Make_id):  
    List = []
    for Year in ME_df.Year:
        List.extend(getMatch(Make_id,Year))
    ME_df['Options'] = List
    return ME_df


# def getHightestScores(Make_id,Year):
#     Scores = []
#     if type(getMatch(Make_id,Year)[0]) !=str:
#         for rec in getMatch(Make_id,Year):
#             Scores.append(int(rec[0].split(': ')[1]))
#     else:
#         Scores='N/A'
#     return Scores




## ISUZU


In [12]:
getModel_Year_MakeID(542,'2000')

,Make_ID,Make_Name,Model_ID,Model_Name
0,542,Isuzu,2817,Trooper
1,542,Isuzu,2818,Rodeo
2,542,Isuzu,2819,VehiCross
3,542,Isuzu,3641,NPR/NPR-HD
4,542,Isuzu,11484,Hombre
5,542,Isuzu,14132,FRR
6,542,Isuzu,14133,NQR
7,542,Isuzu,14291,F6
8,542,Isuzu,14292,F7


In [42]:
Make_id = 542

QC_isuzu = getQCtable_Allyear(Make_id)

In [43]:
QC_isuzu

,Make,Model,Year,Options
0,Isuzu,N-Series,2000,"[NPR/NPR-HD, NQR]"
1,Isuzu,N-Series,2001,"[NPR/NPR-HD, NQR]"
2,Isuzu,N-Series,2002,"[NPR/NPR-HD, NQR]"
3,Isuzu,N-Series,2003,"[NPR/NPR-HD, Ascender, NQR]"
4,Isuzu,N-Series,2004,"[NPR/NPR-HD, Ascender, NQR]"
5,Isuzu,N-Series,2005,"[NPR/NPR-HD, NQR/NRR, Ascender]"
6,Isuzu,N-Series,2006,"[NQR/NRR, Ascender, NPR]"
7,Isuzu,N-Series,2007,"[NQR/NRR, Ascender, NPR]"
8,Isuzu,N-Series,2008,"[NQR/NRR, Ascender, NPR]"
9,Isuzu,N-Series,2009,"[NQR/NRR, NPR]"


In [45]:

QC_isuzu.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\QC_single_report\\QC_N-Series.xlsx',index = False)